In [ ]:
import sys
# use line-buffering for both stdout and stderr
# sys.stdout = open(sys.stdout.fileno(), mode='w', buffering=1)
# sys.stderr = open(sys.stderr.fileno(), mode='w', buffering=1)

import hydra
from omegaconf import OmegaConf
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
import pathlib
from diffusion_policy.workspace.base_workspace import BaseWorkspace
 
import os
import hydra
import torch
from omegaconf import OmegaConf
import pathlib
from torch.utils.data import DataLoader
import copy
import random
import wandb
import tqdm
import numpy as np
import shutil
# from diffusion_policy.workspace.base_workspace import BaseWorkspace
from diffusion_policy.policy.robomimic_lowdim_policy import RobomimicLowdimPolicy
from diffusion_policy.dataset.base_dataset import BaseLowdimDataset
from diffusion_policy.env_runner.base_lowdim_runner import BaseLowdimRunner
from diffusion_policy.common.checkpoint_util import TopKCheckpointManager
from diffusion_policy.common.json_logger import JsonLogger
from diffusion_policy.common.pytorch_util import dict_apply, optimizer_to

from diffusion_policy.policy.robomimic_lowdim_policy import RobomimicLowdimPolicy
from diffusion_policy.policy.robomimic_image_policy import RobomimicImagePolicy
from diffusion_policy.dataset.base_dataset import BaseImageDataset
from diffusion_policy.env_runner.base_image_runner import BaseImageRunner

# from diffusion_policy.workspace.train_diffusion_unet_hybrid_workspace import TrainDiffusionUnetHybridWorkspace
import os
import hydra
import torch
from omegaconf import OmegaConf
import pathlib
from torch.utils.data import DataLoader
import copy
import random
import wandb
import tqdm
import numpy as np
import shutil
from diffusion_policy.workspace.base_workspace import BaseWorkspace
from diffusion_policy.policy.diffusion_unet_hybrid_image_policy import DiffusionUnetHybridImagePolicy
from diffusion_policy.dataset.base_dataset import BaseImageDataset
from diffusion_policy.env_runner.base_image_runner import BaseImageRunner
from diffusion_policy.common.checkpoint_util import TopKCheckpointManager
from diffusion_policy.common.json_logger import JsonLogger
from diffusion_policy.common.pytorch_util import dict_apply, optimizer_to
from diffusion_policy.model.diffusion.ema_model import EMAModel
from diffusion_policy.model.common.lr_scheduler import get_scheduler

OmegaConf.register_new_resolver("eval", eval, replace=True)

In [ ]:
from diffusion_policy.env_runner.robomimic_image_runner import RobomimicImageRunner

In [ ]:
!which python3

In [ ]:
!pip list | grep mujoco

In [ ]:
# mujoco                    2.3.2 (mimicgen in another computer)

In [ ]:
config_path='.'
# config_name = 'image_square_mh_diffusion_policy_cnn_worse.yaml'
config_name = "image_square_mh_diffusion_policy_cnn_g40b30.yaml"
hdf5_filter_key = "g40b30"

In [ ]:
with initialize(version_base=None, config_path=config_path):
    cfg_org = compose(
        config_name=config_name,
        overrides=[
            "hydra.run.dir=data/outputs/${now:%Y.%m.%d}/${now:%H.%M.%S}_${name}_${task_name}",
            "training.seed=42",
            "training.device=cuda:0"
        ],
    )
    print(cfg_org)
    
OmegaConf.resolve(cfg_org)

print('resume: ', cfg_org.training.resume)

In [ ]:
# last_checkpoint_dir = "/home/carl_lab/diffusion_policy/data/outputs/2024.12.13/03.05.17_train_diffusion_unet_hybrid_square_image/"
last_checkpoint_dir = None 

In [ ]:
class TrainDiffusionUnetHybridWorkspace(BaseWorkspace):
    include_keys = ['global_step', 'epoch']

    def __init__(self, cfg: OmegaConf, output_dir=None):
        super().__init__(cfg, output_dir=output_dir)

        # set seed
        seed = cfg.training.seed
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)

        # configure model
        self.model: DiffusionUnetHybridImagePolicy = hydra.utils.instantiate(cfg.policy)

        self.ema_model: DiffusionUnetHybridImagePolicy = None
        if cfg.training.use_ema:
            self.ema_model = copy.deepcopy(self.model)

        # configure training state
        self.optimizer = hydra.utils.instantiate(
            cfg.optimizer, params=self.model.parameters())

        # configure training state
        self.global_step = 0
        self.epoch = 0


In [ ]:
# workspace = TrainDiffusionUnetHybridWorkspace(cfg, output_dir=last_checkpoint_dir)
# # workspace.run()

In [ ]:
# lastest_ckpt_path = workspace.get_checkpoint_path()

In [ ]:
# /home/carl_lab/miniconda3/envs/robodiff/lib/python3.9/site-packages/mujoco_py/binaries/linux/mujoco210/bin/libglewegl.so
# /home/carl_lab/.mujoco/mujoco210/bin/libglewegl.so
# /home/carl_lab/.mujoco/mjpro150/bin/libglewegl.so

In [ ]:
# import os
# os.environ['LD_LIBRARY_PATH'] = os.environ.get('LD_LIBRARY_PATH', '') + ':/home/carl_lab/.mujoco/mujoco210/bin'
# os.environ['LD_LIBRARY_PATH'] = os.environ.get('LD_LIBRARY_PATH', '') + ':/usr/lib/nvidia'
# os.environ['LD_LIBRARY_PATH'] = os.environ.get('LD_LIBRARY_PATH', '') + ':/usr/lib/x86_64-linux-gnu'
# os.environ['LD_LIBRARY_PATH'] = os.environ.get('LD_LIBRARY_PATH', '') + ':/home/carl_lab/.mujoco/mujoco210/bin/libglewegl.so'


In [ ]:
# !export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/home/carl_lab/.mujoco/mujoco210/bin

In [ ]:
# !echo $LD_LIBRARY_PATH

### recreating workspace

In [ ]:
cfg = copy.deepcopy(cfg_org)

seed = cfg.training.seed
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# configure model
model = hydra.utils.instantiate(cfg.policy)

ema_model = None
if cfg.training.use_ema:
    ema_model = copy.deepcopy(model)

# configure training state
optimizer = hydra.utils.instantiate(
    cfg.optimizer, params=model.parameters())

# configure training state
global_step = 0
epoch = 0

### ws.run

In [ ]:
output_dir = "/home/carl_lab/diffusion_policy/data/outputs/custom"

In [ ]:
#TODO
# # resume training  
# if cfg.training.resume:
#     lastest_ckpt_path = get_checkpoint_path()
#     if lastest_ckpt_path.is_file():
#         print(f"Resuming from checkpoint {lastest_ckpt_path}")
#         load_checkpoint(path=lastest_ckpt_path)

In [ ]:
# configure dataset
dataset: BaseImageDataset
dataset = hydra.utils.instantiate(cfg.task.dataset)
assert isinstance(dataset, BaseImageDataset) 

In [ ]:
train_dataloader = DataLoader(dataset, **cfg.dataloader)
normalizer = dataset.get_normalizer()

# configure validation dataset
val_dataset = dataset.get_validation_dataset()
val_dataloader = DataLoader(val_dataset, **cfg.val_dataloader)

In [ ]:
model.set_normalizer(normalizer)
if cfg.training.use_ema:
    ema_model.set_normalizer(normalizer)
    
# configure lr scheduler
lr_scheduler = get_scheduler(
    cfg.training.lr_scheduler,
    optimizer= optimizer,
    num_warmup_steps=cfg.training.lr_warmup_steps,
    num_training_steps=(
        len(train_dataloader) * cfg.training.num_epochs) \
            // cfg.training.gradient_accumulate_every,
    # pytorch assumes stepping LRScheduler every epoch
    # however huggingface diffusers steps it every batch
    last_epoch= global_step-1
)

# configure ema
ema: EMAModel = None
if cfg.training.use_ema:
    ema = hydra.utils.instantiate(
        cfg.ema,
        model= ema_model)

In [ ]:
cfg.task.env_runner

In [ ]:
from diffusion_policy.env_runner.robomimic_image_runner import RobomimicImageRunner

In [ ]:
# configure env
env_runner: BaseImageRunner
env_runner = hydra.utils.instantiate(
    cfg.task.env_runner,
    output_dir=output_dir)
assert isinstance(env_runner, BaseImageRunner)

In [ ]:
# pip install "Cython<3"

In [ ]:
# configure checkpoint
topk_manager = TopKCheckpointManager(
    save_dir=os.path.join(output_dir, 'checkpoints'),
    **cfg.checkpoint.topk
)

# device transfer
device = torch.device(cfg.training.device)
model.to(device)
if ema_model is not None:
    ema_model.to(device)
optimizer_to(optimizer, device)

In [ ]:
cfg.training.debug

In [ ]:
train_sampling_batch = None
log_path = os.path.join(output_dir, 'logs.json.txt')
with JsonLogger(log_path) as json_logger:
    for local_epoch_idx in range(cfg.training.num_epochs):
        step_log = dict()
        # ========= train for this epoch ==========
        train_losses = list()
        with tqdm.tqdm(train_dataloader, desc=f"Training epoch {self.epoch}", 
                leave=False, mininterval=cfg.training.tqdm_interval_sec) as tepoch:
            for batch_idx, batch in enumerate(tepoch):
                # device transfer
                batch = dict_apply(batch, lambda x: x.to(device, non_blocking=True))
                if train_sampling_batch is None:
                    train_sampling_batch = batch

                # compute loss
                raw_loss =  model.compute_loss(batch)
                loss = raw_loss / cfg.training.gradient_accumulate_every
                loss.backward()

                # step optimizer
                if global_step % cfg.training.gradient_accumulate_every == 0:
                    optimizer.step()
                    optimizer.zero_grad()
                    lr_scheduler.step()
                
                # update ema
                if cfg.training.use_ema:
                    ema.step(model)

                # logging
                raw_loss_cpu = raw_loss.item()
                tepoch.set_postfix(loss=raw_loss_cpu, refresh=False)
                train_losses.append(raw_loss_cpu)
                step_log = {
                    'train_loss': raw_loss_cpu,
                    'global_step': global_step,
                    'epoch': epoch,
                    'lr': lr_scheduler.get_last_lr()[0]
                }

                is_last_batch = (batch_idx == (len(train_dataloader)-1))
                if not is_last_batch:
                    # log of last step is combined with validation and rollout
                    json_logger.log(step_log)
                    global_step += 1

                if (cfg.training.max_train_steps is not None) \
                    and batch_idx >= (cfg.training.max_train_steps-1):
                    break

        # at the end of each epoch
        # replace train_loss with epoch average
        train_loss = np.mean(train_losses)
        step_log['train_loss'] = train_loss

        # ========= eval for this epoch ==========
        policy = model
        if cfg.training.use_ema:
            policy = ema_model
        policy.eval()

        # run rollout
        if (epoch % cfg.training.rollout_every) == 0:
            runner_log = env_runner.run(policy)
            # log all
            step_log.update(runner_log)

        # run validation
        if (epoch % cfg.training.val_every) == 0:
            with torch.no_grad():
                val_losses = list()
                with tqdm.tqdm(val_dataloader, desc=f"Validation epoch {epoch}", 
                        leave=False, mininterval=cfg.training.tqdm_interval_sec) as tepoch:
                    for batch_idx, batch in enumerate(tepoch):
                        batch = dict_apply(batch, lambda x: x.to(device, non_blocking=True))
                        loss = model.compute_loss(batch)
                        val_losses.append(loss)
                        if (cfg.training.max_val_steps is not None) \
                            and batch_idx >= (cfg.training.max_val_steps-1):
                            break
                if len(val_losses) > 0:
                    val_loss = torch.mean(torch.tensor(val_losses)).item()
                    # log epoch average validation loss
                    step_log['val_loss'] = val_loss

        # run diffusion sampling on a training batch
        if (epoch % cfg.training.sample_every) == 0:
            with torch.no_grad():
                # sample trajectory from training set, and evaluate difference
                batch = dict_apply(train_sampling_batch, lambda x: x.to(device, non_blocking=True))
                obs_dict = batch['obs']
                gt_action = batch['action']
                
                result = policy.predict_action(obs_dict)
                pred_action = result['action_pred']
                mse = torch.nn.functional.mse_loss(pred_action, gt_action)
                step_log['train_action_mse_error'] = mse.item()
                del batch
                del obs_dict
                del gt_action
                del result
                del pred_action
                del mse
        
        # checkpoint
        if (epoch % cfg.training.checkpoint_every) == 0:
            # checkpointing
            if cfg.checkpoint.save_last_ckpt:
                save_checkpoint()
            if cfg.checkpoint.save_last_snapshot:
                save_snapshot()

            # sanitize metric names
            metric_dict = dict()
            for key, value in step_log.items():
                new_key = key.replace('/', '_')
                metric_dict[new_key] = value
            
            # We can't copy the last checkpoint here
            # since save_checkpoint uses threads.
            # therefore at this point the file might have been empty!
            topk_ckpt_path = topk_manager.get_ckpt_path(metric_dict)

            if topk_ckpt_path is not None:
                save_checkpoint(path=topk_ckpt_path)
        # ========= eval end for this epoch ==========
        policy.train()

        # end of epoch
        # log of last step is combined with validation and rollout
        json_logger.log(step_log)
        global_step += 1
        epoch += 1